In [63]:
import pandas as pd  
import numpy as np  
import statsmodels.api as sm
from numpy import median
from numpy import mean

import os
for dirname, _, filenames in os.walk('/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression 
#from xgboost import XGBClassifier
from sklearn.metrics import log_loss, classification_report 
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns

# https://www.kaggle.com/code/abhirupghosh184098/cirrhosis-predictor-eda-ensemble-nn
# https://www.kaggle.com/code/ashishkumarak/ps3e26-liver-cirrhosis-eda-model

/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes/test.csv
/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes/train_df_clean.csv
/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes/test_df_clean.csv
/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes/train.csv
/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes/sample_submission.csv


In [2]:
train = pd.read_csv("/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes/train_df_clean.csv", 
                      delimiter = ",",
                      header = 0)

test = pd.read_csv("/Users/tavsiev/Desktop/projet_kaggle/projet/classification-multi-classes/test_df_clean.csv", 
                      delimiter = ",",
                      header = 0)

train.shape, test.shape # manque la colonne Status

((15000, 20), (10000, 19))

## Partie Train

In [3]:
train1 = train.copy()
train1.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,1055.0,NaN,54,F,NaN,NaN,NaN,N,1.3,NaN,3.64,NaN,NaN,NaN,NaN,209.0,10.5,3.0,C
1,1,3282.0,Placebo,49,F,N,Y,Y,N,0.7,3.09,3.60,96.0,1142.0,71.3,106.0,240.0,12.4,4.0,C
2,2,1653.0,NaN,56,F,NaN,NaN,NaN,N,2.2,NaN,3.64,NaN,NaN,NaN,NaN,139.0,9.5,2.0,C
3,3,999.0,D-penicillamine,62,F,N,Y,N,N,1.0,4.98,3.35,89.0,1601.0,164.3,85.0,394.0,9.7,3.0,C
4,4,2202.0,NaN,49,F,NaN,NaN,NaN,N,17.2,NaN,3.15,NaN,NaN,NaN,NaN,432.0,11.2,3.0,C


In [4]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15000 non-null  int64  
 1   N_Days         15000 non-null  float64
 2   Drug           8564 non-null   object 
 3   Age            15000 non-null  int64  
 4   Sex            15000 non-null  object 
 5   Ascites        8561 non-null   object 
 6   Hepatomegaly   8563 non-null   object 
 7   Spiders        8555 non-null   object 
 8   Edema          15000 non-null  object 
 9   Bilirubin      15000 non-null  float64
 10  Cholesterol    6669 non-null   float64
 11  Albumin        15000 non-null  float64
 12  Copper         8452 non-null   float64
 13  Alk_Phos       8558 non-null   float64
 14  SGOT           8555 non-null   float64
 15  Tryglicerides  6630 non-null   float64
 16  Platelets      14414 non-null  float64
 17  Prothrombin    14974 non-null  float64
 18  Stage 

In [5]:
for col in train1.select_dtypes(include=['object']).columns:
    unique_values = ', '.join(map(str, train1[col].unique()))
    print(f"Colonne '{col}': {unique_values}")

Colonne 'Drug': nan, Placebo, D-penicillamine
Colonne 'Sex': F, M
Colonne 'Ascites': nan, N, Y
Colonne 'Hepatomegaly': nan, Y, N
Colonne 'Spiders': nan, Y, N
Colonne 'Edema': N, Y, S
Colonne 'Status': C, D, CL


In [6]:
(train1.isnull().sum() / len(train1) * 100).round(2)
#(train.isna().sum() / len(train) * 100).round(2)

id                0.00
N_Days            0.00
Drug             42.91
Age               0.00
Sex               0.00
Ascites          42.93
Hepatomegaly     42.91
Spiders          42.97
Edema             0.00
Bilirubin         0.00
Cholesterol      55.54
Albumin           0.00
Copper           43.65
Alk_Phos         42.95
SGOT             42.97
Tryglicerides    55.80
Platelets         3.91
Prothrombin       0.17
Stage             0.00
Status            0.00
dtype: float64

### Colonnes Object (Mode)

In [7]:
# mode pour les Hommes et Femmes selon la cible Status pour différentes colonnes (ancienne méthode)
# par la suite, Status remplacé par Stage

# DRUG:
# mode pour les Hommes selon la cible Status:
#train[train['Sex'] == 'M'].groupby('Status')['Drug'].agg(lambda x: x.mode().iloc[0])
# mode pour les Femmees selon la cible Status:
#train[train['Sex'] == 'F'].groupby('Status')['Drug'].agg(lambda x: x.mode().iloc[0])

# ASCITES:
# mode pour les Hommes selon la cible Status:
#train[train['Sex'] == 'M'].groupby('Status')['Ascites'].agg(lambda x: x.mode().iloc[0])
# mode pour les Femmes selon la cible Status:
#train[train['Sex'] == 'F'].groupby('Status')['Ascites'].agg(lambda x: x.mode().iloc[0])

# HEPATOMEGALY:
# mode pour les Hommes selon la cible Status:
#train[train['Sex'] == 'M'].groupby('Status')['Hepatomegaly'].agg(lambda x: x.mode().iloc[0])
# mode pour les Femmes selon la cible Status:
#train[train['Sex'] == 'F'].groupby('Status')['Hepatomegaly'].agg(lambda x: x.mode().iloc[0])

# SPIDERS:
# mode pour les Hommes selon la cible Status:
#train[train['Sex'] == 'M'].groupby('Status')['Spiders'].agg(lambda x: x.mode().iloc[0])
# mode pour les Femmes selon la cible Status:
#train[train['Sex'] == 'F'].groupby('Status')['Spiders'].agg(lambda x: x.mode().iloc[0])

#### Drug

In [8]:
# mode pour les Hommes selon la cible Stage:
train1[train1['Sex'] == 'M'].groupby('Stage')['Drug'].agg(lambda x: x.mode().iloc[0])

Stage
1.0            Placebo
2.0    D-penicillamine
3.0    D-penicillamine
4.0    D-penicillamine
Name: Drug, dtype: object

In [9]:
# mode pour les Femmees selon la cible Stage:
train1[train1['Sex'] == 'F'].groupby('Stage')['Drug'].agg(lambda x: x.mode().iloc[0])

Stage
1.0    D-penicillamine
2.0    D-penicillamine
3.0    D-penicillamine
4.0            Placebo
Name: Drug, dtype: object

#### Ascites

In [10]:
# mode pour les Hommes selon la cible Stage:
train1[train1['Sex'] == 'M'].groupby('Stage')['Ascites'].agg(lambda x: x.mode().iloc[0])

Stage
1.0    N
2.0    N
3.0    N
4.0    N
Name: Ascites, dtype: object

In [11]:
# mode pour les Femmes selon la cible Stage:
train1[train1['Sex'] == 'F'].groupby('Stage')['Ascites'].agg(lambda x: x.mode().iloc[0])


Stage
1.0    N
2.0    N
3.0    N
4.0    N
Name: Ascites, dtype: object

#### Hepatomegaly

In [12]:
# mode pour les Hommes selon la cible Stage:
train1[train1['Sex'] == 'M'].groupby('Stage')['Hepatomegaly'].agg(lambda x: x.mode().iloc[0])

Stage
1.0    N
2.0    N
3.0    N
4.0    Y
Name: Hepatomegaly, dtype: object

In [13]:
# mode pour les Femmes selon la cible Stage:
train1[train1['Sex'] == 'F'].groupby('Stage')['Hepatomegaly'].agg(lambda x: x.mode().iloc[0])

Stage
1.0    N
2.0    N
3.0    N
4.0    Y
Name: Hepatomegaly, dtype: object

#### Spiders

In [14]:
# mode pour les Hommes selon la cible Stage:
train1[train1['Sex'] == 'M'].groupby('Stage')['Spiders'].agg(lambda x: x.mode().iloc[0])

Stage
1.0    N
2.0    N
3.0    N
4.0    N
Name: Spiders, dtype: object

In [15]:
# mode pour les Femmes selon la cible Stage:
train1[train1['Sex'] == 'F'].groupby('Stage')['Spiders'].agg(lambda x: x.mode().iloc[0])

Stage
1.0    N
2.0    N
3.0    N
4.0    N
Name: Spiders, dtype: object

In [16]:
# modes_drug = {
#     ('M', 'C'): 'Placebo',
#     ('M', 'CL'): 'D-penicillamine',
#     ('M', 'D'): 'D-penicillamine',
#     ('F', 'C'): 'D-penicillamine',
#     ('F', 'CL'): 'D-penicillamine',
#     ('F', 'D'): 'D-penicillamine'
# }

# modes_ascites = {
#     ('M', 'C'): 'N',
#     ('M', 'CL'): 'N',
#     ('M', 'D'): 'N',
#     ('F', 'C'): 'N',
#     ('F', 'CL'): 'N',
#     ('F', 'D'): 'N'
# }

# modes_hepatomegaly = {
#     ('M', 'C'): 'N',
#     ('M', 'CL'): 'Y',
#     ('M', 'D'): 'Y',
#     ('F', 'C'): 'N',
#     ('F', 'CL'): 'Y',
#     ('F', 'D'): 'Y'
# }

# modes_spiders = {
#     ('M', 'C'): 'N',
#     ('M', 'CL'): 'N',
#     ('M', 'D'): 'N',
#     ('F', 'C'): 'N',
#     ('F', 'CL'): 'N',
#     ('F', 'D'): 'N'
# }

# def na_par_mode(row):
#     key = (row['Sex'], row['Status'])
#     if pd.isna(row['Drug']):
#         row['Drug'] = modes_drug[key]
#     if pd.isna(row['Ascites']):
#         row['Ascites'] = modes_ascites[key]
#     if pd.isna(row['Hepatomegaly']):   

In [17]:
# modes pour chaque colonne selon la modalité 'Stage'
modes_drug = {
    ('M', 1.0): 'Placebo',
    ('M', 2.0): 'Placebo',
    ('M', 3.0): 'Placebo',
    ('M', 4.0): 'D-penicillamine',
    ('F', 1.0): 'D-penicillamine',
    ('F', 2.0): 'D-penicillamine',
    ('F', 3.0): 'D-penicillamine',
    ('F', 4.0): 'D-penicillamine'
}

modes_ascites = {
    ('M', 1.0): 'N',
    ('M', 2.0): 'N',
    ('M', 3.0): 'N',
    ('M', 4.0): 'N',
    ('F', 1.0): 'N',
    ('F', 2.0): 'N',
    ('F', 3.0): 'N',
    ('F', 4.0): 'N'
}

modes_hepatomegaly = {
    ('M', 1.0): 'N',
    ('M', 2.0): 'N',
    ('M', 3.0): 'N',
    ('M', 4.0): 'Y',
    ('F', 1.0): 'N',
    ('F', 2.0): 'N',
    ('F', 3.0): 'N',
    ('F', 4.0): 'Y'
}

modes_spiders = {
    ('M', 1.0): 'N',
    ('M', 2.0): 'N',
    ('M', 3.0): 'N',
    ('M', 4.0): 'N',
    ('F', 1.0): 'N',
    ('F', 2.0): 'N',
    ('F', 3.0): 'N',
    ('F', 4.0): 'N'
}

def na_par_mode(row):
    key = (row['Sex'], row['Stage'])
    if pd.isna(row['Drug']):
        row['Drug'] = modes_drug[key]
    if pd.isna(row['Ascites']):
        row['Ascites'] = modes_ascites[key]
    if pd.isna(row['Hepatomegaly']):
        row['Hepatomegaly'] = modes_hepatomegaly[key]
    if pd.isna(row['Spiders']):
        row['Spiders'] = modes_spiders[key]
    return row

train1 = train1.apply(na_par_mode, axis=1)

### Colonnes Numériques (Médiane)

In [18]:
# la médiane pour chaque colonne numérique, par sexe et par modalité de Stage
medianes = train1.groupby(['Sex', 'Stage'])[['Cholesterol', 'Copper', 'Alk_Phos', 
                                             'SGOT', 'Tryglicerides', 'Platelets', 
                                             'Prothrombin']].median()

medianes

Cholesterol  Copper  Alk_Phos     SGOT  Tryglicerides  Platelets  \
Sex Stage                                                                     
F   1.0          2.730    39.0     911.0   83.700           96.0      269.0   
    2.0          2.810    41.0    1024.0   91.000           94.0      265.0   
    3.0          2.795    42.0    1029.0   89.450           98.0      265.0   
    4.0          2.740    73.0    1138.0  117.800          100.0      210.0   
M   1.0          3.250    74.5    2145.0   86.800           91.0      244.0   
    2.0          3.250    91.5    1083.0  116.250          118.0      269.0   
    3.0          3.240    91.0    1362.0  113.335          113.0      252.0   
    4.0          3.220   111.0    1440.0  121.080           99.5      211.0   

           Prothrombin  
Sex Stage               
F   1.0          10.40  
    2.0          10.40  
    3.0          10.40  
    4.0          11.00  
M   1.0          10.85  
    2.0          10.60  
    3.0          10.60  
    4.0          11.00

In [19]:
moyennes = train1.groupby(['Sex', 'Stage'])[['Cholesterol', 'Copper', 'Alk_Phos', 
                                             'SGOT', 'Tryglicerides', 'Platelets', 
                                             'Prothrombin']].mean().round(2)
moyennes

Cholesterol  Copper  Alk_Phos    SGOT  Tryglicerides  Platelets  \
Sex Stage                                                                    
F   1.0           2.99   46.67   1322.79   91.97         102.66     270.71   
    2.0           3.20   59.19   1569.92   99.92         107.15     270.07   
    3.0           3.17   58.43   1532.20   99.12         108.40     272.38   
    4.0           3.44  100.78   1853.90  119.74         114.57     222.18   
M   1.0           3.30   79.00   2660.78  108.16          91.56     242.59   
    2.0           3.53  109.38   2115.54  112.75         120.84     267.40   
    3.0           3.72  110.89   2235.34  122.68         120.54     256.13   
    4.0           3.78  146.28   2338.36  122.12         119.84     213.88   

           Prothrombin  
Sex Stage               
F   1.0          10.37  
    2.0          10.44  
    3.0          10.41  
    4.0          11.00  
M   1.0          10.91  
    2.0          10.65  
    3.0          10.66  
    4.0          11.12

In [20]:
colonnes_a_traiter = ['Cholesterol', 'Copper', 'Alk_Phos', 
                       'SGOT', 'Tryglicerides', 'Platelets', 
                       'Prothrombin']

def na_par_mediane(row):
    for col in colonnes_a_traiter:
        if pd.isna(row[col]):
            row[col] = medianes.loc[(row['Sex'], row['Stage']), col]
    return row

train1 = train1.apply(na_par_mediane, axis=1)

In [21]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15000 non-null  int64  
 1   N_Days         15000 non-null  float64
 2   Drug           15000 non-null  object 
 3   Age            15000 non-null  int64  
 4   Sex            15000 non-null  object 
 5   Ascites        15000 non-null  object 
 6   Hepatomegaly   15000 non-null  object 
 7   Spiders        15000 non-null  object 
 8   Edema          15000 non-null  object 
 9   Bilirubin      15000 non-null  float64
 10  Cholesterol    15000 non-null  float64
 11  Albumin        15000 non-null  float64
 12  Copper         15000 non-null  float64
 13  Alk_Phos       15000 non-null  float64
 14  SGOT           15000 non-null  float64
 15  Tryglicerides  15000 non-null  float64
 16  Platelets      15000 non-null  float64
 17  Prothrombin    15000 non-null  float64
 18  Stage 

#### Normalisation avec PowerTransformer: pour rendre les données plus gaussiennes

In [22]:
train2 = train1.copy()
# Normalisation avec PowerTransformer: pour rendre les données plus gaussienne
colonnes_to_normalize = ['Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 
                        'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']

transformer = PowerTransformer(method = 'yeo-johnson')
train2[colonnes_to_normalize] = transformer.fit_transform(train2[colonnes_to_normalize])

#### Encoding

In [23]:
for col in train2.select_dtypes(include=['object']).columns:
    unique_values = ', '.join(map(str, train2[col].unique()))
    print(f"Colonne '{col}': {unique_values}")

Colonne 'Drug': D-penicillamine, Placebo
Colonne 'Sex': F, M
Colonne 'Ascites': N, Y
Colonne 'Hepatomegaly': N, Y
Colonne 'Spiders': N, Y
Colonne 'Edema': N, Y, S
Colonne 'Status': C, D, CL


In [24]:
train2['Stage'][:3]

0    3.0
1    4.0
2    2.0
Name: Stage, dtype: float64

In [25]:
# Custom Mapping pour Stage car l'ordre a l'air d'avoir son importance 
# This encoding is particularly useful for ordinal variable where the order of categories is important
# To make sure that the learning algorithm interprets the ordinal variables correctly, 
# we can map the categorical values to integer values manually. 
print(train2['Stage'].unique())  
train2['Stage'] = train2['Stage'].map({1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3})
print(train2['Stage'].unique())

[3. 4. 2. 1.]
[2 3 1 0]


In [26]:
# OneHot encoding pour les autres
# Liste des colonnes catégoriques
colonnes_cat = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

train_encoded = pd.get_dummies(train2, columns=colonnes_cat, drop_first=False)
train_encoded.head()

,id,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,...,Sex_M,Ascites_N,Ascites_Y,Hepatomegaly_N,Hepatomegaly_Y,Spiders_N,Spiders_Y,Edema_N,Edema_S,Edema_Y
0,0,1055.0,54,0.394125,-0.029251,0.276397,-0.331315,-0.127848,-0.243023,-0.005883,...,False,True,False,True,False,True,False,True,False,False
1,1,3282.0,49,-0.705394,0.452011,0.165050,0.956386,0.103139,-0.916440,0.298979,...,False,True,False,False,True,False,True,True,False,False
2,2,1653.0,56,1.095203,-0.002295,0.276397,-0.370399,-0.138923,-0.192454,-0.172167,...,False,True,False,True,False,True,False,True,False,False
3,3,999.0,62,-0.044473,2.124549,-0.509516,0.842839,0.779310,1.507787,-0.586686,...,False,True,False,False,True,True,False,True,False,False
4,4,2202.0,49,1.972027,-0.029251,-1.022634,-0.331315,-0.127848,-0.243023,-0.005883,...,False,True,False,True,False,True,False,True,False,False


In [27]:
train_encoded.columns

Index(['id', 'N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper',
       'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin',
       'Stage', 'Status', 'Drug_D-penicillamine', 'Drug_Placebo', 'Sex_F',
       'Sex_M', 'Ascites_N', 'Ascites_Y', 'Hepatomegaly_N', 'Hepatomegaly_Y',
       'Spiders_N', 'Spiders_Y', 'Edema_N', 'Edema_S', 'Edema_Y'],
      dtype='object')

In [28]:
# la colonne cible avec Label Encoding
label_encoder = LabelEncoder()
train_encoded['Status_encoded'] = label_encoder.fit_transform(train_encoded['Status']) 
train_encoded.drop('Status', axis=1, inplace=True)
train_encoded['Status_encoded'].value_counts()

Status_encoded
0    10049
2     4560
1      391
Name: count, dtype: int64

In [29]:
# le modèle 
X = train_encoded.drop(['id', 'Status_encoded'], axis=1) 
y = train_encoded['Status_encoded']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=10000)
# solver='lbfgs' par défaut

model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)

logloss = log_loss(y_test, y_pred_proba)
print(f"Log Loss: {logloss}")

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['C', 'CL', 'D']))

/Users/tavsiev/Desktop/projet_kaggle/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Log Loss: 0.4373056808432266
              precision    recall  f1-score   support

           C       0.85      0.92      0.88      2006
          CL       0.00      0.00      0.00        74
           D       0.78      0.69      0.73       920

    accuracy                           0.83      3000
   macro avg       0.54      0.54      0.54      3000
weighted avg       0.81      0.83      0.81      3000



/Users/tavsiev/Desktop/projet_kaggle/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Autre méthode